In [29]:
import numpy as np
import cv2
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import matplotlib.pyplot as plt

In [30]:
dir_path ='D:/Multi_Label/'

In [31]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [32]:
len(img_list)

3646

In [33]:
train=ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.1,zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)
test=ImageDataGenerator(rescale=1/255,validation_split=0.1)
train_generator=train.flow_from_directory(dir_path,target_size=(300,300),batch_size=32,
                                          class_mode='categorical',subset='training')
test_generator=test.flow_from_directory(dir_path,target_size=(300,300),batch_size=32,
                                        class_mode='categorical',subset='validation')
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 3327 images belonging to 52 classes.
Found 349 images belonging to 52 classes.
{0: '고철', 1: '고철_비닐', 2: '고철_유리병', 3: '고철_유리병_비닐', 4: '고철_유리병_음식물', 5: '고철_유리병_플라스틱', 6: '고철_음식물', 7: '고철_플라스틱', 8: '고철_플라스틱_비닐', 9: '비닐', 10: '비닐_스티로폼', 11: '비닐_스티로폼_음식물', 12: '비닐_음식물', 13: '스티로폼', 14: '스티로폼_음식물', 15: '유리병', 16: '유리병_비닐', 17: '유리병_플라스틱', 18: '음식물', 19: '일반쓰레기', 20: '일반쓰레기_고철', 21: '일반쓰레기_고철_유리병', 22: '일반쓰레기_고철_플라스틱', 23: '일반쓰레기_비닐', 24: '일반쓰레기_유리병', 25: '일반쓰레기_유리병_플라스틱_음식물', 26: '일반쓰레기_음식물', 27: '일반쓰레기_종이', 28: '일반쓰레기_종이_비닐', 29: '일반쓰레기_종이_음식물', 30: '일반쓰레기_종이_플라스틱', 31: '일반쓰레기_플라스틱', 32: '종이', 33: '종이_고철', 34: '종이_고철_유리병', 35: '종이_고철_유리병_음식물', 36: '종이_고철_유리병_플라스틱', 37: '종이_고철_플라스틱', 38: '종이_비닐', 39: '종이_비닐_음식물', 40: '종이_스티로폼', 41: '종이_유리병', 42: '종이_유리병_플라스틱', 43: '종이_음식물', 44: '종이_플라스틱_비닐_음식물', 45: '종이_플라스틱_음식물', 46: '종이_플리스틱', 47: '캔_플라스틱', 48: '플라스틱', 49: '플라스틱_비닐', 50: '플라스틱_비닐_음식물', 51: '플라스틱_음식물'}


In [34]:
'''
model=Sequential()
    
model.add(Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(52,activation='softmax'))
'''
model = Sequential()

model.add(Conv2D(32, kernel_size=5, strides=2, activation='relu', input_shape=(300, 300, 3)))
model.add(Conv2D(64, kernel_size=3, strides=1, activation='relu'))       

model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='sigmoid'))   # Final Layer using Sigmoid

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
filepath="trained_model.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

In [35]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      2432      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
dense_7 (Dense)              (None, 146, 146, 128)     8320      
_________________________________________________________________
dense_8 (Dense)              (None, 146, 146, 52)      6708      
Total params: 35,956
Trainable params: 35,956
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [37]:
hist = model.fit_generator(train_generator, epochs=100, steps_per_epoch=2276//32,validation_data=test_generator,
                    validation_steps=251//32,callbacks=callbacks_list)

Epoch 1/100


ValueError: Error when checking target: expected dense_8 to have 4 dimensions, but got array with shape (32, 52)

In [ ]:
#학습 상태를 그래프로 그리기
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()

In [ ]:
#학습결과 저장하기
model.save_weights('Trash_Classification_CNN-weight.h5')

In [ ]:
#학습결과 불러오기
model.load_weights('Trash_Classification_CNN-weight.h5')